In [1]:
import os
import faiss
import tqdm
import json
import numpy as np
from utils import extract_features_single, load_pca, create_model_and_tokenizer

from datasets import load_dataset

# model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_name = "facebook/contriever-msmarco"
model, tokenizer = create_model_and_tokenizer(model_name)
model = model.cuda()

query_data = load_dataset('json', data_files='./lm-eval-train.jsonl', split='train')

data = load_dataset('json', data_files='/root/data/wiki_en.jsonl', split='train')


Loading tokenizer...

Loading model...

In [2]:
vecs = np.load(f'./features/{model_name}/wiki_en.npy')

In [3]:
new_data = []
new_vecs = []
for i, item in enumerate(data):
    if item['text'].split().__len__() < 256:
        continue
    new_data.append(item)
    new_vecs.append(vecs[i])

In [4]:
len(vecs)

6630651

In [5]:
len(new_vecs)

2961873

In [6]:
vecs = np.stack(new_vecs, 0)

In [8]:
data = new_data

In [9]:
def build_index(
    xb: np.ndarray,
    d: int = None,
):
    if d is None:
        d = xb.shape[-1]
    res = faiss.StandardGpuResources()
    # index = faiss.index_factory(d, "IVF100,PQ8")
    # index = faiss.index_factory(d, "HNSW32")
    
    # Param of PQ
    M = 16  # The number of sub-vector. Typically this is 8, 16, 32, etc.
    nbits = 8 # bits per sub-vector. This is typically 8, so that each sub-vec is encoded by 1 byte
    # Param of IVF
    nlist = 10000  # The number of cells (space partition). Typical value is sqrt(N)
    # Param of HNSW
    hnsw_m = 32  # The number of neighbors for HNSW. This is typically 32

    # Setup
    quantizer = faiss.IndexHNSWFlat(d, hnsw_m)
    index = faiss.IndexIVFPQ(quantizer, d, nlist, M, nbits)
    
    # Sample 1_000_000 vectors to train the index.
    # xt = xb[np.random.choice(xb.shape[0], 1000_000, replace=False)]
    xt = xb
    index.train(xt)
    # index.add(xb)
    return index

def add_to_index(
    xb,
    index,
):
    index.add(xb)
    return index

In [10]:
def search(queries, k=100):
    features = extract_features_single(queries, model, tokenizer)
    _, ids = index.search(features, k=k)
    return [data[int(i)] for i in ids[0]]

In [11]:
index = build_index(vecs)

In [12]:
index = add_to_index(vecs, index)

In [13]:
search("Where is Zurich?", k=10)

[{'text': 'Basel ( , ), also known as Basle ( ), is a city in northwestern Switzerland on the river Rhine. Basel is Switzerland\'s third-most-populous city (after Zürich and Geneva) with about 175,000 inhabitants. The official language of Basel is (the Swiss variety of Standard) German, but the main spoken language is the local Basel German dialect.\n\nBasel is commonly considered to be the cultural capital of Switzerland and the city is famous for its many museums, including the Kunstmuseum, which is the first collection of art accessible to the public in the world (1661) and the largest museum of art in Switzerland, the Fondation Beyeler (located in Riehen), the Museum Tinguely and the Museum of Contemporary Art, which is the first public museum of contemporary art in Europe. Forty museums are spread throughout the city-canton, making Basel one of the largest cultural centres in relation to its size and population in Europe.\n\nThe University of Basel, Switzerland\'s oldest universit

In [16]:
with open(f'/root/ft_data/lm_aug_wiki_hnsw_ivf_256.jsonl', 'w') as f:
    for query in tqdm.tqdm(query_data):
        items = search(query['text'])
        for rank, item in enumerate(items):
            item['meta']['augment'] = {
                'rank': rank,
                'query_meta': query['meta'],
            }
            # item['meta'].pop('timestamp')
            f.write(json.dumps(item) + '\n')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79195/79195 [46:31<00:00, 28.37it/s]


In [ ]:
exit()

In [ ]:
1